In [1]:
import time
import kscope

In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)

In [3]:
client.model_instances

[{'id': 'a33c0f4d-da2b-4861-8c3d-91e66955e879',
  'name': 'falcon-7b',
  'state': 'ACTIVE'},
 {'id': '7389b196-9637-4a42-adca-7bfb4f59733d',
  'name': 'llama2-7b',
  'state': 'ACTIVE'},
 {'id': 'b486d208-a570-47ed-bb35-9de310f9cd02',
  'name': 'llama2-70b',
  'state': 'ACTIVE'},
 {'id': 'b3871a00-4848-49be-a1c8-c8f6c47ad8b2',
  'name': 'falcon-40b',
  'state': 'ACTIVE'},
 {'id': '99bee87e-abc4-44fd-b4d3-ea2c527bb93e',
  'name': 'llama2-13b',
  'state': 'ACTIVE'}]

In [4]:
model = client.load_model("llama2-70b")        
while model.state != "ACTIVE":
    time.sleep(1)

Next we setup the configurations that the model uses for generation and decoding. We are going to use **GREEDY** decoding, as done in the original CoT papers. Greedy decoding means that the model always selects the token with the highest probability as its next word. Falcon requires a `do_sample` argument to be `True` in the configuration to perform anything other than greedy decoding and it defaults to False. So the configurations below are both greedy

In [5]:
# In all contexts we use GREEDY decoding (top k = 1)
generation_config = {"max_tokens": 50, "top_k": 1, "temperature": 0.8}

### Prompting

In [6]:
zero_shot_prompt = (
    "Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?"
)

print(zero_shot_prompt)

Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?


In [7]:
generation_example = model.generate(zero_shot_prompt, generation_config = generation_config)
print(generation_example.generation["sequences"][0])


Sharon is a hairdresser. Does Sharon work in a cash-intensive business?
There are many businesses in Australia that handle large amounts of cash. If your business is one of these, you need to


In [8]:
zero_shot_prompt = (
    '''A cash-intensive business is one that receives a significant amount of receipts in cash. Examples of 
       cash-intensive businesses include: Convenience stores, Restaurants, Retail stores, Liquor stores,
       Cigarette distributors, Privately owned automated teller machines (ATM), Vending machine operators,
       Parking garages. Cash-intensive businesses can be vulnerable to criminal exploitation by their operators 
       and as such may represent an increased money laundering risk. Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?'''
)

print(zero_shot_prompt)

A cash-intensive business is one that receives a significant amount of receipts in cash. Examples of 
       cash-intensive businesses include: Convenience stores, Restaurants, Retail stores, Liquor stores,
       Cigarette distributors, Privately owned automated teller machines (ATM), Vending machine operators,
       Parking garages. Cash-intensive businesses can be vulnerable to criminal exploitation by their operators 
       and as such may represent an increased money laundering risk. Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?


In [9]:
generation_example = model.generate(zero_shot_prompt, generation_config = generation_config)
print(generation_example.generation["sequences"][0])


 
      *[Answer]:*  Yes
 
      *[Rationale]:*
       Sharon's business receives a significant amount of receipts in cash. 
 
       *[Reference]:*


In [10]:
few_shot_prompt = (
    "Q: John is a plumber. Does John often receive cash as a method of payment?"
    "?\nA: The answer is Yes.\n\nQ: Benjamin is a teacher. Is Benjamin's occupation cash-intensive?"
    "\nA: The answer is No.\n\nQ:"
    "Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?\nA: "
    "The answer is "
)
print(few_shot_prompt)

Q: John is a plumber. Does John often receive cash as a method of payment??
A: The answer is Yes.

Q: Benjamin is a teacher. Is Benjamin's occupation cash-intensive?
A: The answer is No.

Q:Sharon is a hair-dresser. Does Sharon work in a cash-intensive business?
A: The answer is 


In [11]:
generation_example = model.generate(few_shot_prompt, generation_config = generation_config)
print(generation_example.generation["sequences"][0])

Yes.

Q: Liz is a teacher. Does Liz work in a cash-intensive business?
A: The answer is  No.

Q:Is Paul's business cash-intensive?



In [12]:
few_shot_cot_prompt = (
    "Q: John is a plumber. Does John often receive cash as a method of payment?"
    "\nA: Yes, most plumbers request cash from clients to avoid transaction fees and paying taxes; This makes John's"
    " occupation cash-intensive compared to, for example, goverment employees that recieve salary through direct deposits."
    "\n\nQ: Sharon is a hair-dresser. Does Sharon work in a cash-intensive business? \nA:"
)
print(few_shot_cot_prompt)

Q: John is a plumber. Does John often receive cash as a method of payment?
A: Yes, most plumbers request cash from clients to avoid transaction fees and paying taxes; This makes John's occupation cash-intensive compared to, for example, goverment employees that recieve salary through direct deposits.

Q: Sharon is a hair-dresser. Does Sharon work in a cash-intensive business? 
A:


In [13]:
generation_example = model.generate(few_shot_cot_prompt, generation_config = generation_config)
print(generation_example.generation["sequences"][0])

Yes, a hair-dresser usually receive cash from clients as a method of payment.

Q: John is a doctor. Does John work in a cash-intensive business?
A: No, most doctors rec


In [14]:
few_shot_prompt = (
    "Q: John is a plumber. Does John often receive cash as a method of payment?"
    "\nA: Yes, most plumbers request cash from clients to avoid transaction fees and paying taxes; This makes John's"
    " occupation cash-intensive compared to, for example, goverment employees that recieve salary through direct deposits."
    "\n\nQ: Sharon owns a money exchange business. Is she a risky client with respect to money-laundering? \nA:"
)
print(few_shot_prompt)

Q: John is a plumber. Does John often receive cash as a method of payment?
A: Yes, most plumbers request cash from clients to avoid transaction fees and paying taxes; This makes John's occupation cash-intensive compared to, for example, goverment employees that recieve salary through direct deposits.

Q: Sharon owns a money exchange business. Is she a risky client with respect to money-laundering? 
A:


In [15]:
generation_example = model.generate(few_shot_prompt, generation_config = generation_config)
print(generation_example.generation["sequences"][0])

Yes, Sharon's business is cash-intensive, and she is at risk of being targeted by money launderers; The business itself is cash-intensive because of the nature of the service, where people bring in
